### Download Flowers dataset

In [0]:
#You can download the data manually as well instead of using 'wget'
!wget http://download.tensorflow.org/example_images/flower_photos.tgz --quiet

In [0]:
#Read the dataset
import tarfile
dataset = tarfile.open('flower_photos.tgz')

In [0]:
#We will build a pandas dataset
import pandas as pd
df = pd.DataFrame(columns=['class','image_file'])

In [0]:
#Run through tarfile members 
for name in dataset.getnames():
    
    tar_mem = dataset.getmember(name)
    
    #Check if it is a file
    if(tar_mem.isfile() and name.endswith('.jpg')):
        #Build directory and class info
        im_dir = name[0:name.rfind('/')]
        im_class = im_dir[im_dir.rfind('/')+1:]
        #Add record to the dataframe
        df.loc[df.shape[0]] = [im_class, name]

In [0]:
#extract data
dataset.extractall(path='')

In [7]:
!ls -l flower_photos

total 608
drwx------ 2 270850 5000  36864 Feb 10  2016 daisy
drwx------ 2 270850 5000  49152 Feb 10  2016 dandelion
-rw-r----- 1 270850 5000 418049 Feb  9  2016 LICENSE.txt
drwx------ 2 270850 5000  36864 Feb 10  2016 roses
drwx------ 2 270850 5000  36864 Feb 10  2016 sunflowers
drwx------ 2 270850 5000  40960 Feb 10  2016 tulips


Create Training & Test Dataset

In [0]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [0]:
train_df.to_csv('flower_photos/train.csv',index=False)
test_df.to_csv('flower_photos/test.csv', index=False)

### Read training and test data

In [0]:
#Read training and test Dataframe
train_df = pd.read_csv('flower_photos/train.csv')
test_df = pd.read_csv('flower_photos/test.csv')

In [6]:
#Check contents
train_df.sample(n=5)

,class,image_file
1700,sunflowers,flower_photos/sunflowers/23645265812_24352ff6b...
1416,tulips,flower_photos/tulips/3454461550_64d6e726bf_m.jpg
2007,roses,flower_photos/roses/6241886381_cc722785af.jpg
1914,roses,flower_photos/roses/8442304572_2fdc9c7547_n.jpg
2456,sunflowers,flower_photos/sunflowers/9460336948_6ae968be93...


In [22]:
#Get class names
class_names = train_df['class'].unique().tolist()
print('Flower classes: ', class_names)

Flower classes:  ['tulips', 'daisy', 'sunflowers', 'dandelion', 'roses']


### Build Batch generator (using ImageDataGenerator)

In [0]:
import tensorflow as tf
import numpy as np

In [0]:
#Define some parameters
img_size = 224
img_depth = 3  

Function to normalize image according to Model being used

In [0]:
def normalize_data(img):
    
    #Normalize for MobileNet
    return tf.keras.applications.resnet50.preprocess_input(img)

Defime ImageDataGenerator for both Training and Test Separately

In [0]:
#Define Training Data Generator with augmentations
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=20,
                                                                width_shift_range=0.2,
                                                                height_shift_range=0.2,
                                                                horizontal_flip=True,
                                                                preprocessing_function=normalize_data) #Normalize the data accordingly

#Define Test Data Generator with NO augmentations
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=normalize_data) #Normalize the data accordingly

Create Data Generators objects for Training and Test

In [12]:
#Training (from dataframe)
train_generator = train_datagen.flow_from_dataframe(train_df, 
                                                    x_col='image_file', #File path for image
                                                    y_col='class',           #Class for the image
                                                    target_size=(img_size, img_size), #Image resize dimensions
                                                    batch_size=64)

Found 2936 validated image filenames belonging to 5 classes.


In [13]:
#Test (from dataframe)
test_generator = test_datagen.flow_from_dataframe(test_df,
                                                  x_col='image_file', #File path for image
                                                  y_col='class',           #Class for the image
                                                  target_size=(img_size, img_size), #Image resize dimensions
                                                  batch_size=64)

Found 734 validated image filenames belonging to 5 classes.


ImageDataGenerator has lot of useful features. Learn more about ImageDataGenerator at https://keras.io/preprocessing/image/

### Load pre-trained model

In [14]:
tf.keras.backend.clear_session()
model = tf.keras.applications.ResNet50(include_top=False, #Do not include classification layer for imagenet
                                       input_shape=(img_size,img_size, img_depth),
                                       weights='imagenet')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [15]:
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [16]:
model.output

<tf.Tensor 'conv5_block3_out/Relu:0' shape=(?, 7, 7, 2048) dtype=float32>

Freeze the layers in Pre-trained model

In [0]:
#Set pre-trained model layers to not trainable
for layer in model.layers:
    layer.trainable = False

In [18]:
#Check if layers frozen
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

### Add FC layer for new classes

In [0]:
#get Output layer of Pre0trained model
x = model.output

#Global average pool to reduce number of features and Flatten the output
x = tf.keras.layers.GlobalAveragePooling2D()(x)

In [20]:
#Output shape of Global Average Pooling
x

<tf.Tensor 'global_average_pooling2d/Mean:0' shape=(?, 2048) dtype=float32>

In [0]:
#Add output layer
prediction = tf.keras.layers.Dense(len(class_names),activation='softmax')(x)

### Building final model for Classification

In [0]:
#Using Keras Model class
final_model = tf.keras.models.Model(inputs=model.input, #Pre-trained model input as input layer
                                    outputs=prediction) #Output layer added

In [0]:
#Compile the model
final_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [26]:
#How does our overall model looks
final_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

### Train the model

In [0]:
#Saving the best model using model checkpoint callback
model_checkpoint=tf.keras.callbacks.ModelCheckpoint('flowers_resnet.h5', 
                                                    save_best_only=True, 
                                                    monitor='val_acc', 
                                                    mode='max', 
                                                    verbose=1)

In [28]:
final_model.fit_generator(train_generator, 
                          epochs=5,
                          steps_per_epoch= 2936//64,
                          validation_data=test_generator,
                          validation_steps = 734//64, 
                          callbacks=[model_checkpoint])

Epoch 1/5
11/45 [======>.......................] - ETA: 12s - loss: 0.6684 - acc: 0.7571
Epoch 00001: val_acc improved from -inf to 0.75710, saving model to flowers_resnet.h5
45/45 [==============================] - 51s 1s/step - loss: 0.7737 - acc: 0.7086 - val_loss: 0.6684 - val_acc: 0.7571
Epoch 2/5
11/45 [======>.......................] - ETA: 12s - loss: 0.5279 - acc: 0.8125
Epoch 00002: val_acc improved from 0.75710 to 0.81250, saving model to flowers_resnet.h5
45/45 [==============================] - 40s 893ms/step - loss: 0.3845 - acc: 0.8625 - val_loss: 0.5279 - val_acc: 0.8125
Epoch 3/5
11/45 [======>.......................] - ETA: 13s - loss: 0.4836 - acc: 0.8253
Epoch 00003: val_acc improved from 0.81250 to 0.82528, saving model to flowers_resnet.h5
45/45 [==============================] - 40s 896ms/step - loss: 0.3079 - acc: 0.8938 - val_loss: 0.4836 - val_acc: 0.8253
Epoch 4/5
11/45 [======>.......................] - ETA: 13s - loss: 0.4657 - acc: 0.8324
Epoch 00004: val_

In [29]:
#Lets train for 5 more steps
final_model.fit_generator(train_generator, 
                          epochs=10,
                          initial_epoch=5,
                          steps_per_epoch= 2936//64,
                          validation_data=test_generator,
                          validation_steps = 734//64, 
                          callbacks=[model_checkpoint])

Epoch 6/10
11/45 [======>.......................] - ETA: 10s - loss: 0.5406 - acc: 0.8253
Epoch 00006: val_acc did not improve from 0.83239
45/45 [==============================] - 40s 883ms/step - loss: 0.2251 - acc: 0.9262 - val_loss: 0.5406 - val_acc: 0.8253
Epoch 7/10
11/45 [======>.......................] - ETA: 11s - loss: 0.4309 - acc: 0.8509
Epoch 00007: val_acc improved from 0.83239 to 0.85085, saving model to flowers_resnet.h5
45/45 [==============================] - 41s 900ms/step - loss: 0.2110 - acc: 0.9251 - val_loss: 0.4309 - val_acc: 0.8509
Epoch 8/10
11/45 [======>.......................] - ETA: 12s - loss: 0.5039 - acc: 0.8281
Epoch 00008: val_acc did not improve from 0.85085
45/45 [==============================] - 39s 868ms/step - loss: 0.2044 - acc: 0.9269 - val_loss: 0.5039 - val_acc: 0.8281
Epoch 9/10
11/45 [======>.......................] - ETA: 12s - loss: 0.4296 - acc: 0.8509
Epoch 00009: val_acc did not improve from 0.85085
45/45 [============================

### Unfreeze some of Trained Layers in ResNet

In [30]:
print(len(model.layers))

175


Let's unfreeze 10% at the end (which have high end features more specific to ImageNet)

In [0]:
#Unfreezing all layers after layer# 158
for layer in model.layers[158:]:
    layer.trainable = True    

In [36]:
#We will need to recompile the model
final_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
final_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [37]:
#Lets train for 10 steps
final_model.fit_generator(train_generator, 
                          epochs=20,
                          initial_epoch=10,
                          steps_per_epoch= 2936//64,
                          validation_data=test_generator,
                          validation_steps = 734//64, 
                          callbacks=[model_checkpoint])

Epoch 11/20
11/45 [======>.......................] - ETA: 13s - loss: 21.9090 - acc: 0.6264
Epoch 00011: val_acc did not improve from 0.85085
45/45 [==============================] - 43s 948ms/step - loss: 0.3282 - acc: 0.9025 - val_loss: 21.9090 - val_acc: 0.6264
Epoch 12/20
11/45 [======>.......................] - ETA: 11s - loss: 2.0108 - acc: 0.7898
Epoch 00012: val_acc did not improve from 0.85085
45/45 [==============================] - 40s 889ms/step - loss: 0.1349 - acc: 0.9558 - val_loss: 2.0108 - val_acc: 0.7898
Epoch 13/20
11/45 [======>.......................] - ETA: 12s - loss: 0.5272 - acc: 0.8949
Epoch 00013: val_acc improved from 0.85085 to 0.89489, saving model to flowers_resnet.h5
45/45 [==============================] - 41s 911ms/step - loss: 0.1026 - acc: 0.9697 - val_loss: 0.5272 - val_acc: 0.8949
Epoch 14/20
11/45 [======>.......................] - ETA: 12s - loss: 0.5902 - acc: 0.8665
Epoch 00014: val_acc did not improve from 0.89489
45/45 [======================

In [38]:
#Lets train for 10 more steps
final_model.fit_generator(train_generator, 
                          epochs=30,
                          initial_epoch=20,
                          steps_per_epoch= 2936//64,
                          validation_data=test_generator,
                          validation_steps = 734//64, 
                          callbacks=[model_checkpoint])

Epoch 21/30
11/45 [======>.......................] - ETA: 11s - loss: 0.6256 - acc: 0.8679
Epoch 00021: val_acc did not improve from 0.89489
45/45 [==============================] - 41s 911ms/step - loss: 0.0302 - acc: 0.9920 - val_loss: 0.6256 - val_acc: 0.8679
Epoch 22/30
11/45 [======>.......................] - ETA: 12s - loss: 1.0379 - acc: 0.8395
Epoch 00022: val_acc did not improve from 0.89489
45/45 [==============================] - 40s 892ms/step - loss: 0.0254 - acc: 0.9937 - val_loss: 1.0379 - val_acc: 0.8395
Epoch 23/30
11/45 [======>.......................] - ETA: 13s - loss: 0.7387 - acc: 0.8608
Epoch 00023: val_acc did not improve from 0.89489
45/45 [==============================] - 41s 911ms/step - loss: 0.0190 - acc: 0.9951 - val_loss: 0.7387 - val_acc: 0.8608
Epoch 24/30
11/45 [======>.......................] - ETA: 13s - loss: 0.7722 - acc: 0.8537
Epoch 00024: val_acc did not improve from 0.89489
45/45 [==============================] - 40s 890ms/step - loss: 0.0309

At this point, our model is overfit. How do we improve our model when using Transfer Learning. Here are some approaches to try:

1. Unfreeze lesser number of layers (fewer parameters to train)
2. Train unfrozen layer with smaller learning rate (avoiding big changes to weights)
3. Use Dropout before output layer